# Long Document Content Extraction

We often need GPT-3 to help us extract key figures, dates or other bits of important content from documents that are too big to fit into the context window. One approach for solving this is to chunk the document up and process each chunk separately, before combining into one list of answers. 

In this notebook we'll run through this approach:
- We'll load in a long PDF and pull the text out
- We'll create a prompt to be used to extract key bits of information
- We'll chunk up our document and process each chunk to pull any answers out
- We'll then combine them at the end
- This simple approach will then be extended to three more difficult questions

## Setup

We'll take our PDF, a Formula 1 Financial Regulation document on Power Units, and extract the text from it for entity extraction. We'll use this to try to extract answers that are buried in the content.

In [ ]:
!pip install textract

In [3]:
import textract
import os
import openai

# Extract the raw text from each PDF using textract
text = textract.process('data/fia_f1_power_unit_financial_regulations_issue_1_-_2022-08-16.pdf', method='pdfminer').decode('utf-8')
clean_text = text.replace("  ", " ").replace("\n", "; ").replace(';',' ')

## Simple Entity Extraction

We'll start by extracting key bits of information from chunks of a document.

We'll accomplish this by:
- Creating a template prompt with our questions and an example of the format it expects
- Create a function to take a chunk of text as input, combine with the prompt and get a response
- Run a script to chunk the text, extract answers and output them for parsing

In [4]:
# Example prompt - 
document = '<document>'
template_prompt=f'''Extract key pieces of information from this regulation document. 
If a particular piece of information is not present, output \"Not specified\". 
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Author\n1. What is the amount of the power unit cost cap in USD, GBP and EUR\n2. What is the value of External Manufacturing Costs in USD\n3. What is the Capital Expenditure Limit in USD\n\nDocument: \"\"\"{document}\"\"\"\n\n0. Author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

Extract key pieces of information from this regulation document. 
If a particular piece of information is not present, output "Not specified". 
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Author
1. What is the amount of the power unit cost cap in USD, GBP and EUR
2. What is the value of External Manufacturing Costs in USD
3. What is the Capital Expenditure Limit in USD

Document: """<document>"""

0. Author: Tom Anderson (Page 1)
1.


In [5]:
import os
import openai


def extract_chunk(document,template_prompt):
    
    prompt=template_prompt.replace('<document>',document)

    response = openai.Completion.create(
    model='text-davinci-003', 
    prompt=prompt,
    temperature=0,
    max_tokens=1500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return "1." + response['choices'][0]['text']

In [6]:
chunks = []
results = []

# chunk up the contract into consecutive 10k character chunks
for i in range(0, len(clean_text), 10000):
    #print(clean_text[i:i+10000])
    chunks.append(clean_text[i:i+10000])

for chunk in chunks:
    results.append(extract_chunk(chunk,template_prompt))
    print(results[-1])


1. What is the amount of the power unit cost cap in USD, GBP and EUR: USD 95,000,000 (Page 3) GBP 76,459,000 (Page 4) EUR 90,210,000 (Page 4)
2. What is the value of External Manufacturing Costs in USD: Not specified
3. What is the Capital Expenditure Limit in USD: Not specified
1. What is the amount of the power unit cost cap in USD, GBP and EUR: Not specified (Page 1)
2. What is the value of External Manufacturing Costs in USD: Not specified (Page 1)
3. What is the Capital Expenditure Limit in USD: Not specified (Page 1)
1. What is the amount of the power unit cost cap in USD, GBP and EUR: Not specified (Page 8)
2. What is the value of External Manufacturing Costs in USD: US Dollars 20,000,000 in respect of each of the Full Year Reporting Periods ending on 31 December 2023, 31 December 2024 and 31 December 2025, adjusted for Indexation (Page 10)
3. What is the Capital Expenditure Limit in USD: Not specified (Page 10)
1. What is the amount of the power unit cost cap in USD, GBP and EU

In [7]:
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. What is the amount of the power unit cost cap in USD, GBP and EUR: USD 95,000,000 (Page 3) GBP 76,459,000 (Page 4) EUR 90,210,000 (Page 4)',
 '2. What is the value of External Manufacturing Costs in USD: US Dollars 20,000,000 in respect of each of the Full Year Reporting Periods ending on 31 December 2023, 31 December 2024 and 31 December 2025, adjusted for Indexation (Page 10)',
 '3. What is the Capital Expenditure Limit in USD: US Dollars 30,000,000 (Page 32)']

## Complex Entity Extraction

Now we'll try to answer more difficult questions that need tougher logic to work out.

We'll update the prompt with three more questions and see whether our model can work them out.

In [8]:
# Example prompt - 
template_prompt=f'''Extract key pieces of information from this regulation document. 
If a particular piece of information is not present, output \"Not specified\". 
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Author\n1. How is a minor overspend breach calculated\n2. How is a major overspend breach calculated\n3.Which years do these financial regulations apply to\n\nDocument: \"\"\"{document}\"\"\"\n\n0. Author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

Extract key pieces of information from this regulation document. 
If a particular piece of information is not present, output "Not specified". 
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Author
1. How is a minor overspend breach calculated
2. How is a major overspend breach calculated
3.Which years do these financial regulations apply to

Document: """<document>"""

0. Author: Tom Anderson (Page 1)
1.


In [9]:
chunks = []
results = []

for i in range(0, len(clean_text), 10000):
    chunks.append(clean_text[i:i+10000])

for chunk in chunks:
    results.append(extract_chunk(chunk,template_prompt))
    
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. How is a minor overspend breach calculated: Relevant Costs reported exceed the Power Unit Cost Cap by less than 5% (Page 24)',
 '2. How is a major overspend breach calculated: Relevant Costs reported exceed the Power Unit Cost Cap by more than 5% (Page 24)',
 '3. Which years do these financial regulations apply to: 2023-2026 (Page 2)',
 '3. Which years do these financial regulations apply to: 2022 (Page 4)',
 '3. Which years do these financial regulations apply to: 2022 (Page 8)',
 '3. Which years do these financial regulations apply to: 2022 (Page 1)',
 '3. Which years do these financial regulations apply to: 2022 (Page 1)',
 '3.Which years do these financial regulations apply to: 2022 (Page 1)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 onwards (Page 34)',
 '3.Which years do these financial regulations apply to: 2022 (Page 1)',
 '3.Which years do these financial regulations apply to: 2026 onwards (Page 41)',
 '3.Which years do these financi

## Consolidation

We've been able to extract the first two answers safely, while the third was confounded by the date that appeared on every page, though the correct answer is in there as well.

To tune this further you can consider experimenting with:
- A more descriptive or specific prompt
- Negative prompting, telling the model to ignore a particular thing i.e. the footnote date on every page
- If you have sufficient training data, fine-tuning a model to find a set of outputs very well
- The way you chunk your data - we have gone for 10,000 characters with no overlap, but more intelligent chunking that breaks info into sections, cuts by tokens or similar may get better results

However, with minimal tuning we have now answered 6 questions of varying difficulty using the contents of a long document, and have a reusable approach that we can apply to any long document requiring entity extraction. Look forward to seeing what you can do with this!